In [1]:
%run ~/work/jupyter_notebooks/gene\ family\ distances/correlate_evolution.ipynb

%cd ~/work/clusterEvo/new_tests/archaea/

/nobackup1b/users/thiberio/clusterEvo/new_tests/archaea


In [2]:
from scipy.spatial.distance import braycurtis
from IPython.display        import HTML, display, clear_output
from Bio                    import AlignIO, SeqIO, Align, Alphabet
from copy                   import deepcopy

import igraph     as ig
import numpy      as np
import seaborn    as sns
import pandas     as pd
import colorlover as cl

import itertools
import multiprocessing
import random
import os
import subprocess
import re
import ete3

In [3]:
ncbi         = ete3.NCBITaxa()
aln_alphabet = Alphabet.Gapped(Alphabet.IUPAC.ambiguous_dna)

In [4]:
class cd:
    """
    Context manager for changing the current working directory
    """
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [5]:
graph                 = ig.Graph.Read_Picklez('coevolving_graph.pkl')
minimum_size_clusters = []
for clst_num in set(graph.vs['louvain']):
    clst_nodes = graph.vs.select(louvain     =clst_num, 
#                                  trusted_clst=True
                                )
    if len(clst_nodes) < 10:# or clst_num < 0:
        continue
    
#     if np.median(clst_nodes['num_taxa']) < 30:
#         continue
    
    print(clst_num, len(clst_nodes))
    minimum_size_clusters.append(clst_num)

0 35
1 15
2 47
3 111
4 62
5 28
8 18
15 15


In [6]:
Counter(graph.vs.select(
    louvain_in=minimum_size_clusters
)['louvain']).most_common()

[(3, 111), (4, 62), (2, 47), (0, 35), (5, 28), (8, 18), (1, 15), (15, 15)]

In [9]:
dist_matrices = []
group_names   = []

for clst_num in minimum_size_clusters:
    clst_nodes = graph.vs.select(louvain     =clst_num,
#                                  trusted_clst=True
                                )
    
    for group_num in clst_nodes['name']:
        try:
            tmp_matrix = pd.read_csv(f'matrices/{group_num}.mldist', 
                                      delim_whitespace = True, 
                                      skiprows         = 1, 
                                      header           = None,
                                      index_col        = 0)
        except FileNotFoundError:
            print(f'*Missing gene family {group_num}')
            continue

        convert_table = {}
        for seq_name in re.findall('>(\S+)', 
                                   open(f'minimum_size_groups/{group_num}.faa').read()):
            seq_name = seq_name.split('|')
            convert_table['_'.join(seq_name)] = f'{seq_name[1]}|{seq_name[0]}'

        tmp_matrix.rename(index  =convert_table, 
                          inplace=True)
        tmp_matrix.columns = tmp_matrix.index

        dist_matrices.append(tmp_matrix.copy())
        group_names.append(  group_num)
    
table = dict(zip(group_names, dist_matrices))

In [10]:
def get_taxa(group_name, matrix):
    taxa = [taxon.split('|')[0] for taxon in matrix.index]
    return( group_name, taxa )

group_taxa = pd.DataFrame( columns=['group_num', 'taxa'],
                           data   =[get_taxa(group, table[group]) 
                                    for group in table.keys()] )
group_taxa.set_index('group_num', inplace=True)

all_genomes = set().union(*group_taxa.taxa.values)

In [ ]:
for clst_num in minimum_size_clusters:
# for clst_num in [4]:

    print(clst_num)

    if not os.path.isfile(f'refined_cluster_phylogenies/cluster_{clst_num}_matching_copies.parquet'):
        clst_subgraph = graph.vs.select(louvain     =clst_num,
                                        trusted_clst=True).subgraph()
        
        if np.median(clst_subgraph.vs['num_taxa']) < 30:
            continue

        clst_subgraph.vs.select(clst_degree=0).delete()

        matching_copies = pd.DataFrame(columns=clst_subgraph.vs['name'],
                                       index  =all_genomes)
        matching_copies = matching_copies.applymap(lambda x: [])

        iteration_size = clst_subgraph.ecount()
        for count, edge in enumerate(clst_subgraph.es):
            group1 = edge.source_vertex['name']
            group2 = edge.target_vertex['name']

            tmp_matrix1 = table[group1].copy()
            tmp_matrix2 = table[group2].copy()

            matrix1, taxa1, matrix2, taxa2 = balance_matrices(tmp_matrix1.copy(), 
                                                              tmp_matrix2.copy(), 
                                                              gene_sep   ='|')

            for (index1, row1), (index2, row2) in zip(taxa1.iterrows(), taxa2.iterrows()):
                matching_copies.loc[row1.genome, group1].append(row1.gene)
                matching_copies.loc[row2.genome, group2].append(row2.gene)

            print('\rIteration '+str(round((count/iteration_size)*100, 2))+'%', end='')

        count += 1
        print('\rIteration '+str(round((count/iteration_size)*100, 2))+'%', end='')

        matching_copies.rename(columns=dict( zip(matching_copies.columns, 
                                                 matching_copies.columns.astype(str)) ),
                               inplace=True)
        matching_copies.to_parquet(f'refined_cluster_phylogenies/cluster_{clst_num}_matching_copies.parquet')
    
    print()

In [ ]:
for clst_num in minimum_size_clusters:
    clst_nodes = graph.vs.select(louvain=clst_num)
            
    print(clst_num)

    if not os.path.isfile(f'cluster_phylogenies/cluster_{clst_num}_matching_copies.parquet'):
        clst_subgraph = graph.vs.select(louvain=clst_num).subgraph()
        clst_subgraph.vs.select(clst_degree=0).delete()

        matching_copies = pd.DataFrame(columns=clst_subgraph.vs['name'],
                                       index  =all_genomes)
        matching_copies = matching_copies.applymap(lambda x: [])

        iteration_size = clst_subgraph.ecount()
        for count, edge in enumerate(clst_subgraph.es):
            group1 = edge.source_vertex['name']
            group2 = edge.target_vertex['name']

            tmp_matrix1 = table[group1].copy()
            tmp_matrix2 = table[group2].copy()

            matrix1, taxa1, matrix2, taxa2 = balance_matrices(tmp_matrix1.copy(), 
                                                              tmp_matrix2.copy(), 
                                                              gene_sep   ='|')

            for (index1, row1), (index2, row2) in zip(taxa1.iterrows(), taxa2.iterrows()):
                matching_copies.loc[row1.genome, group1].append(row1.gene)
                matching_copies.loc[row2.genome, group2].append(row2.gene)

            print('\rIteration '+str(round((count/iteration_size)*100, 2))+'%', end='')

        count += 1
        print('\rIteration '+str(round((count/iteration_size)*100, 2))+'%', end='')

        matching_copies.rename(columns=dict( zip(matching_copies.columns, 
                                                 matching_copies.columns.astype(str)) ),
                               inplace=True)
        matching_copies.to_parquet(f'cluster_phylogenies/cluster_{clst_num}_matching_copies.parquet')
    
    print()

In [ ]:
best_copy_supports = []
for clst_num in minimum_size_clusters:
    
    matching_copies = pd.read_parquet(f'cluster_phylogenies/cluster_{clst_num}_matching_copies.parquet')

    to_drop = matching_copies.columns.intersection(graph.vs.select(single_copy=True)['name'])
    matching_copies.drop(columns=to_drop, 
                         inplace=True)

    matching_copy_freq = matching_copies.applymap(   lambda x: Counter(x))
    best_copies        = matching_copy_freq.applymap(lambda x: x.most_common()[0][0] 
                                                               if x 
                                                               else np.nan)
    tmp_support = matching_copy_freq.applymap(lambda cell: 
                                              max(cell.values()) / sum(cell.values()) if cell 
                                              else np.nan)
    
    best_copy_supports.extend( list(itertools.chain(*tmp_support.values)) )

best_copy_supports = np.array(best_copy_supports)
best_copy_supports = best_copy_supports[pd.notna(best_copy_supports)]

In [ ]:
np.percentile(best_copy_supports, 3.5), np.median(best_copy_supports)

In [ ]:
number_of_genomes_with_duplications = []
number_of_genomes                   = []
for node in graph.vs:
    
    group_num = node['name']
    with open(f'minimum_size_groups/{group_num}.faa') as fasta_handle:
        
        group_genomes = re.findall('^>[^|]+?\|(.*)$', fasta_handle.read(), re.M)
        if len(group_genomes) == len(set(group_genomes)):
            continue
            
        number_of_genomes.append(node['num_taxa'])
        
        duplicated_genomes_count = 0
        for genome, copy_count in Counter(group_genomes).most_common():
            if copy_count == 1:
                break
            duplicated_genomes_count += 1
        
        number_of_genomes_with_duplications.append( duplicated_genomes_count )
    
number_of_genomes_with_duplications = np.array(number_of_genomes_with_duplications)
number_of_genomes                   = np.array(number_of_genomes)

In [ ]:
419-len(number_of_genomes_with_duplications)

In [ ]:
number_of_genomes_with_duplications[(number_of_genomes_with_duplications <= 2) & (number_of_genomes >= 35)].shape

In [ ]:
number_of_genomes_with_duplications[number_of_genomes >= 35].shape

In [ ]:
number_of_genomes_with_duplications[number_of_genomes_with_duplications <= 2].shape

In [ ]:
len(graph.vs.select(louvain    =4, 
                    single_copy=True,
                    num_taxa_ge=35
                   )
   )

In [ ]:
len(graph.vs.select(louvain    =4, 
                    single_copy=False,
#                     num_taxa_ge=35
                   )
   )

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=100)

sns.kdeplot(
    number_of_genomes_with_duplications, 
#     bw   =1, 
    shade=True, 
    ax   =ax);

ax.set_title('# genomes containing multiple copies')
ax.set_xlim(0)
sns.despine(offset=10)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=100)

sns.kdeplot(
    best_copy_supports, 
    bw   =0.02, 
    shade=True, 
    ax   =ax);

ax.set_title('Within cluster confidence in most frequent paralog')
sns.despine(offset=10)

In [ ]:
def generate_fasta(column, target_folder):
    header2seq = {}
    with open(f'minimum_size_groups/{column[0]}.faa') as fasta:
        for line in fasta:
            if line.startswith('>'):
                header              = line.strip('>\n ')
                header2seq[header]  = ''
            else:
                line                = line.strip(' \n')
                header2seq[header] += line.replace('-', '')
    
    with open(f'{target_folder}/{column[0]}.faa', 'w') as fasta:
        for genome_acc, protein_id in column[1].items():
            if pd.isna(protein_id):
                continue

            seq = header2seq[f'{protein_id}|{genome_acc}']
            fasta.write(f'>{genome_acc}\n{seq}\n')

    return(column[0])

In [ ]:
def run_alignment(input_arg):
    group_num, clst_num = input_arg
    with open(f'refined_cluster_phylogenies/cluster_{clst_num}_alignments/{group_num}.aln', 'w') as aln_out:
        subprocess.call(['mafft', 
                         '--auto',
                         '--reorder', 
                         '--quiet', 
                         f'refined_cluster_phylogenies/cluster_{clst_num}_alignments/{group_num}.faa'], 
                        stdout=aln_out)

In [ ]:
def concate_genes(group_numbers, clst_num):

    concat_genomes = set().union(*group_taxa.loc[group_numbers, 'taxa'].values)
    
    missing_genes = {} # just to keep track of the number of missing marker genes in each genome
    concatenation = {}
    for genome in concat_genomes:
        missing_genes[genome]             = 0
        concatenation[genome]             = Align.SeqRecord( Align.Seq('', aln_alphabet) )
        concatenation[genome].name        = genome
        concatenation[genome].id          = genome
        concatenation[genome].description = genome
    
    total_genes      = 0.0 # keep track of the number of genes added to the concatenation
    current_position = 1
    aln_folder       = f'refined_cluster_phylogenies/cluster_{clst_num}_alignments'
    partitions       = open(f'refined_cluster_phylogenies/cluster_{clst_num}', 'w')
    for group_num in group_numbers:

        tmp_aln      = AlignIO.read( f'{aln_folder}/{group_num}.aln', 'fasta' )
        aln_length   = tmp_aln.get_alignment_length() # get the expected size of the alignment so you can compare if all have the same size
        total_genes += aln_length

        genomes_found = set()
        for entry in tmp_aln:
            # if this alignment has a different size from the rest, something is reaaaaaly wrong!
            if len(entry) != aln_length:
                print('\t**Error, block "%s" has a different length than the rest of the MSA: %s' %(entry.name, aln))
            
            concatenation[entry.name] += deepcopy(entry.seq)
            genomes_found.add(entry.name)

        partition_name = f'cluster{group_num}'
        partitions.write(f'LG, {partition_name} = {current_position}-{current_position+aln_length-1}\n')
        current_position += aln_length

        #
        # add gaps for those genomes missing this gene (same size as the expected alignment)
        for genome in concat_genomes.difference(genomes_found):
            concatenation[genome] += Align.Seq( '-' * aln_length, aln_alphabet )
            missing_genes[genome] += aln_length
    partitions.close()

    AlignIO.write( Align.MultipleSeqAlignment( concatenation.values() ), 
                  f'refined_cluster_phylogenies/cluster_{clst_num}.aln', 'fasta' )

In [ ]:
for clst_num in minimum_size_clusters:
    print(clst_num)

    matching_copies = pd.read_parquet(f'refined_cluster_phylogenies/cluster_{clst_num}_matching_copies.parquet')

    matching_copy_freq = matching_copies.applymap(   lambda x: Counter(x))
    best_copies        = matching_copy_freq.applymap(lambda x: x.most_common()[0][0] 
                                                               if x 
                                                               else np.nan)
    
    if not os.path.isdir(f'refined_cluster_phylogenies/cluster_{clst_num}_alignments'):
        os.mkdir(f'refined_cluster_phylogenies/cluster_{clst_num}_alignments')
    
    aln_func_input = []
    for column in best_copies.iteritems():
        generate_fasta(column, 
                       target_folder=f'refined_cluster_phylogenies/cluster_{clst_num}_alignments')
        aln_func_input.append( (column[0], clst_num) )
    
    pool     = multiprocessing.Pool( processes=15 )
    ran_alns = pool.map(run_alignment, aln_func_input)
    pool.close()
    pool.join()
    
#     clst_nodes    = graph.vs.select(louvain=clst_num)
    clst_subgraph = graph.vs.select(louvain     =clst_num,
                                    trusted_clst=True).subgraph()
    clst_subgraph.vs['degree'] = clst_subgraph.degree()
    clst_subgraph.vs.select(degree=0).delete()

    concate_genes(clst_subgraph.vs['name'], clst_num)

#     break

In [11]:
header = 'assembly_accession bioproject biosample wgs_master refseq_category taxid species_taxid \
          organism_name infraspecific_name isolate version_status assembly_level release_type \
          genome_rep seq_rel_date asm_name submitter gbrs_paired_asm paired_asm_comp ftp_path \
          excluded_from_refseq relation_to_type_material'.split()

genbank_summary = pd.read_csv('~/work/assembly_summary_genbank.txt', 
                              sep      ='\t', 
                              index_col=0, 
                              header   =None, 
                              names    =header, 
                              comment  ='#')
genbank_summary = genbank_summary.reindex(index=all_genomes).copy()

In [ ]:
for clst_num in minimum_size_clusters:
    with open(f'cluster_phylogenies/cluster_{clst_num}.treefile') as tree_handle:
        newick  = tree_handle.read()
        
    for index, row in genbank_summary.iterrows():
        newick = newick.replace(index, 
                                str(row.taxid))
    
    with open(f'cluster_phylogenies/cluster_{clst_num}.taxid.treefile', 'w') as tree_handle:
        tree_handle.write(newick)

In [13]:
for clst_num in [2, 3, 4, 5]:
    with open(f'cluster_phylogenies/cluster_{clst_num}-C60.treefile') as tree_handle:
        newick  = tree_handle.read()
        
    for index, row in genbank_summary.iterrows():
        newick = newick.replace(index, 
                                str(row.taxid))
    
    with open(f'cluster_phylogenies/cluster_{clst_num}-C60.taxid.treefile', 'w') as tree_handle:
        tree_handle.write(newick)

In [15]:
with open(f'single_copy_phylogenies/core_genes_from_cluster_all-C60.treefile') as tree_handle:
    newick  = tree_handle.read()

for index, row in genbank_summary.iterrows():
    newick = newick.replace(index, 
                            str(row.taxid))

with open(f'single_copy_phylogenies/core_genes_from_cluster_all-C60.taxid.treefile', 'w') as tree_handle:
    tree_handle.write(newick)

In [12]:
# for clst_num in minimum_size_clusters:
for clst_num in [3, 4, 5]:
    with open(f'refined_cluster_phylogenies/cluster_{clst_num}-C20.treefile') as tree_handle:
        newick  = tree_handle.read()
        
    for index, row in genbank_summary.iterrows():
        newick = newick.replace(index, 
                                str(row.taxid))
    
    with open(f'refined_cluster_phylogenies/cluster_{clst_num}-C20.taxid.treefile', 'w') as tree_handle:
        tree_handle.write(newick)